In [1]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [2]:
perf_dir = "/home/shtoshni/Research/events/models/perf"
slurm_id_list = ["6408270", "6408624"]  #"6407936"

files = []

for slurm_id in slurm_id_list:
    files.extend(sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), 
                                  key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1])))
    
# files = sorted(files, key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

50


In [3]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [4]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [5]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [6]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [7]:
idx = perf_df.groupby(['use_srl', 'use_local_attention', 'srl_loss_wt', 'seed'])['fscore'].idxmax()
dev_max = perf_df.iloc[idx]
dev_max

,use_local_attention,use_srl,sample_invalid,srl_loss_wt,seed,fscore,test_f
15,False,False,0.50,1.0,2,48.2,48.3
1,False,False,0.25,1.0,3,47.1,48.9
17,False,False,0.50,1.0,4,47.8,47.8
18,False,False,0.50,1.0,5,47.3,47.0
4,False,False,0.25,1.0,6,48.1,49.1
20,True,False,0.50,1.0,2,48.2,49.2
6,True,False,0.25,1.0,3,48.8,49.1
7,True,False,0.25,1.0,4,48.4,46.9
8,True,False,0.25,1.0,5,49.3,48.7
9,True,False,0.25,1.0,6,49.0,48.2


In [11]:
perf_df.groupby(['use_srl', 'use_local_attention', 'srl_loss_wt', 'sample_invalid'])[['fscore', 'test_f']].mean()


fscore  test_f
use_srl use_local_attention srl_loss_wt sample_invalid                
False   False               1.0         0.25             47.32   48.74
                                        0.50             47.26   47.94
        True                1.0         0.25             48.64   48.26
                                        0.50             48.26   48.90
True    False               0.1         0.25             47.84   49.28
                                        0.50             47.72   48.76
                            0.5         0.25             47.80   48.24
                                        0.50             47.04   48.56
                            1.0         0.25             47.62   48.88
                                        0.50             47.38   48.72